# FCT Unesp - TCC Treino 1
Aluno: Pétrus Antonio Bárbosa Pradella

TCC2 - Treinando análise preditiva para dados de transações comerciais.

Teste 1
    - DataSet Utilizado: Weekly Sales Transactions
    - Fonte: Kaggle
    - Link: https://www.kaggle.com/crawford/weekly-sales-transactions
    
    Dataset disponibilizado pelo "UC Irvine Machine Learning Repository"

Caracteristicas gerais do Dataset:

    - Número de Instancias: 811
    - Número de Atributos: 53
    - Estatisticas de Uso em 29/08/2019 (kaggle):
        24,992 views
        3,112 downloads

    - Mais detalhes tecnicos podem ser encontrados em:
        https://archive.ics.uci.edu/ml/datasets/Sales_Transactions_Dataset_Weekly


### Importando as bibliotecas necessárias

In [1]:
import numpy as np
import pandas as pd
import math


from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split



In [2]:
#Configurações para as análises algébricas:

autocorr_nlags_max = 13 # Maximum number of lags for calculating autocorrelation
limiar_de_correlacao = 0.7 # Use >= 0.7 for strongly correlated
param_top_perc = 0.05 #Porcentagem limiar para escolher os melhores produtos, se aumentar esse valor mais produtos serão consideradores como "produtos populares"

### Carregando a base de dados

In [3]:
current_folder = "C:/Users/Petrus/Desktop/UNESP/Docs 2016-2020/2019/Segundo Semestre/TCC2/ProjetoTCC2/Treino1";
dataset_file = current_folder + '/Sales_Transactions_Dataset_Weekly.csv';
dataset = pd.read_csv(dataset_file)

In [4]:
dataset.head(5)

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,Normalized 42,Normalized 43,Normalized 44,Normalized 45,Normalized 46,Normalized 47,Normalized 48,Normalized 49,Normalized 50,Normalized 51
0,P1,11,12,10,8,13,12,14,21,6,...,0.06,0.22,0.28,0.39,0.50,0.00,0.22,0.17,0.11,0.39
1,P2,7,6,3,2,7,1,6,3,3,...,0.20,0.40,0.50,0.10,0.10,0.40,0.50,0.10,0.60,0.00
2,P3,7,11,8,9,10,8,7,13,12,...,0.27,1.00,0.18,0.18,0.36,0.45,1.00,0.45,0.45,0.36
3,P4,12,8,13,5,9,6,9,13,13,...,0.41,0.47,0.06,0.12,0.24,0.35,0.71,0.35,0.29,0.35
4,P5,8,5,13,11,6,7,9,14,9,...,0.27,0.53,0.27,0.60,0.20,0.20,0.13,0.53,0.33,0.40


### Filtrando os atributos Normalizados

In [5]:
col_names = dataset.columns

print(col_names)

Index(['Product_Code', 'W0', 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8',
       ...
       'Normalized 42', 'Normalized 43', 'Normalized 44', 'Normalized 45',
       'Normalized 46', 'Normalized 47', 'Normalized 48', 'Normalized 49',
       'Normalized 50', 'Normalized 51'],
      dtype='object', length=107)


In [6]:
#dataset_filtered = pd.read_csv(dataset_file).filter(regex='Product_Code|^W[0-9]+$')

#Tambem dá para fazer via definição direta... mas o regexa ali é bem mais facil quando se tem como validar ele em algum outro lugar.
dataset_filtered = pd.read_csv(dataset_file, 
                       usecols=[col for col in col_names if 'Product_Code' in col or 'W' in col],
                       index_col = 'Product_Code')

print(dataset_filtered.shape)
dataset_filtered.head(5)

(811, 52)


,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
Product_Code,,,,,,,,,,,,,,,,,,,,,
P1,11,12,10,8,13,12,14,21,6,14,...,4,7,8,10,12,3,7,6,5,10
P2,7,6,3,2,7,1,6,3,3,3,...,2,4,5,1,1,4,5,1,6,0
P3,7,11,8,9,10,8,7,13,12,6,...,6,14,5,5,7,8,14,8,8,7
P4,12,8,13,5,9,6,9,13,13,11,...,9,10,3,4,6,8,14,8,7,8
P5,8,5,13,11,6,7,9,14,9,9,...,7,11,7,12,6,6,5,11,8,9


In [7]:
#n_produtos = dataset_filtered[['Product_Code']].count().sum();  # Dá erro pois Product_code virou um index_de_coluna e não mais uma coluna separada!

n_produtos = dataset_filtered.shape[0]; #Como fizemos a removeção da coluna de Product_code não dá mais para contar ela, contudo o número de produtos é o mesmo que o número de linhas!
n_semanas = len(dataset_filtered.columns);  #Se não tivesse removido a primeira coluna, deveria ser [nColumns - 1];

print("Número de Produtos Cadastrados: ", n_produtos)
print("Número de Semanas Contabilizadas: ", n_semanas)


Número de Produtos Cadastrados:  811
Número de Semanas Contabilizadas:  52


### Aplicando lógica de regressão linear! (Ténicas usada "Spearman")

In [8]:
#Aplicando o método Spearman
# Mais informações Tecnicas: https://kite.com/python/examples/665/scipy-compute-the-spearman-correlation
# Mais informações Pedagógicas: https://statistics.laerd.com/statistical-guides/spearmans-rank-order-correlation-statistical-guide.php

#A ténicas Spearman é recomendada para esse caso pois trata-se de um regressão linear para um dataset que possui apenas dois atributos a serem analisados.

correlacao_produtos = dataset_filtered.transpose().corr(method='spearman')
print(correlacao_produtos.shape)
correlacao_produtos.head(5)



(811, 811)


Product_Code,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P810,P811,P812,P813,P814,P815,P816,P817,P818,P819
Product_Code,,,,,,,,,,,,,,,,,,,,,
P1,1.000000,0.105156,-0.033156,-0.060961,0.024067,-0.115742,0.034288,-0.083970,0.254553,0.178971,...,0.010585,0.141629,0.145374,-0.089562,-0.111400,0.132112,-0.193239,0.021788,-0.246475,-0.009766
P2,0.105156,1.000000,0.051913,0.139827,-0.098849,0.139774,0.023473,0.192785,0.054797,-0.061026,...,0.118540,-0.086185,-0.032924,-0.098239,-0.116745,0.011166,-0.048712,-0.114477,-0.115255,0.098174
P3,-0.033156,0.051913,1.000000,0.339249,-0.003896,-0.078649,-0.018139,-0.226969,-0.023219,0.073690,...,-0.324464,-0.033638,-0.240631,0.162646,-0.045399,-0.225324,-0.080792,-0.246736,0.037838,0.120155
P4,-0.060961,0.139827,0.339249,1.000000,0.030796,0.071470,0.064682,-0.130114,-0.117943,-0.154483,...,0.050968,-0.311405,-0.127180,0.280844,-0.005571,0.029330,0.131983,0.040744,-0.254841,0.245973
P5,0.024067,-0.098849,-0.003896,0.030796,1.000000,-0.096208,0.143222,0.020834,0.100637,0.316596,...,-0.047638,-0.006176,-0.032229,0.115919,0.284669,0.095391,-0.087876,0.108401,0.013865,-0.263292


### Procurando valores com coeficiente alto!


In [9]:
set_de_coeficiente_alto = set() # record the individual Product_Code values involved
pares_de_coeficiente_alto = [] # record the Product_Code pairs and the correlation value

for row in correlacao_produtos:
    for col in correlacao_produtos.columns:
        if row < col:
            
            valor_de_correlacao = correlacao_produtos.loc[row, col]

            if abs(valor_de_correlacao) >= limiar_de_correlacao:
                # Se o valor é maior que o limiar, salvar ele!
                #print(row)
                #print(col)
                #print(corr_val)
                pares_de_coeficiente_alto.append((row, col, valor_de_correlacao))
                #Adiciona a linha e a coluna
                set_de_coeficiente_alto.add(row) 
                set_de_coeficiente_alto.add(col)
            
#print(high_corr_list)
# Convert list to a dataframe
high_correlation_dataframe = pd.DataFrame(pares_de_coeficiente_alto, columns=['Product_1', 'Product_2', 'Corr'])
print(f"Número de produtos com coeficientes de correlação >= {limiar_de_correlacao} ou <= -{limiar_de_correlacao}: {high_correlation_dataframe.shape[0]}")
print(f"Número total de produtos: {len(set_de_coeficiente_alto)}")
high_correlation_dataframe.head(5)

Número de produtos com coeficientes de correlação >= 0.7 ou <= -0.7: 14
Número total de produtos: 20


,Product_1,Product_2,Corr
0,P18,P191,0.769559
1,P215,P279,0.700140
2,P215,P656,0.700140
3,P251,P253,0.700140
4,P251,P468,0.700140


### Analisando popularidade dos itens (maior e menor venda)


In [10]:
n_produtos = dataset_filtered.shape[0];
n_semanas = len(dataset_filtered.columns);

print("Número de Produtos Cadastrados: ", n_produtos)
print("Número de Semanas Contabilizadas: ", n_semanas)
print(' ')
print(' ')

num_top = math.floor(param_top_perc * n_produtos)
print(f'Encontrar os [{num_top}] produtos mais e menos populares com base na média das vendas semanáis:')

media_venda_mensal = dataset_filtered.mean(axis=1)

produtos_mais_vendidos = media_venda_mensal.nlargest(n=num_top).index.values.tolist()
produtos_menos_vendidos = media_venda_mensal.nsmallest(n=num_top).index.values.tolist()

dataset_mais_vendidos = dataset_filtered[dataset_filtered.index.isin(produtos_mais_vendidos)]
dataset_menos_vendidos = dataset_filtered[dataset_filtered.index.isin(produtos_menos_vendidos)]

Número de Produtos Cadastrados:  811
Número de Semanas Contabilizadas:  52
 
 
Encontrar os [40] produtos mais e menos populares com base na média das vendas semanáis:


In [11]:

print(f"Foram encontrados os [{num_top}] produtos mais populares: {dataset_mais_vendidos.shape[0]}")
dataset_mais_vendidos.head(5)

Foram encontrados os [40] produtos mais populares: 40


,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
Product_Code,,,,,,,,,,,,,,,,,,,,,
P15,19,45,47,42,29,44,43,36,25,52,...,38,35,33,30,37,30,30,34,38,20
P16,30,27,27,43,29,32,49,41,49,38,...,40,30,33,39,42,45,31,31,23,22
P24,36,42,27,33,40,48,38,39,41,39,...,38,36,39,32,31,39,23,40,32,14
P27,44,34,33,39,34,30,47,27,45,39,...,40,33,32,26,26,29,44,30,34,28
P34,47,42,24,55,42,23,41,51,45,29,...,29,25,38,46,46,34,34,45,41,26


In [12]:

print(f"Foram encontrados os [{num_top}] produtos menos populares: {dataset_menos_vendidos.shape[0]}")
dataset_menos_vendidos.head(5)

Foram encontrados os [40] produtos menos populares: 40


,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
Product_Code,,,,,,,,,,,,,,,,,,,,,
P215,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
P230,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P232,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
P250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
